In [ ]:
def generator_fast_sequence(config=None, is_training=True,
              background_ratio=3./8,
              random_scale=np.array([0.5, 1, 1.5, 2.0]),
              vis=False):
    video_set = []
    for root, dirs, files in os.walk(FLAGS.data_path):
        for file in files:
            if file.endswith('.mp4'):
                video_set.append(os.path.splitext(file)[0])
    if is_training:
        index = np.arange(0, len(video_set)-3)
        video_set = video_set[0:(len(video_set)-3)]
    else:
        index = np.arange(0, 3)
        video_set = video_set[(len(video_set)-3):]
    images_array_list = []
    score_maps_array_list = []
    geo_maps_array_list = []
    training_masks_array_list = []
    frame_info = []
    polys_array_list2 = []
    tags_array_list2 = []
    for k in index:
        t_start = time.time()
        xml_solo_path = FLAGS.data_path + '/' + video_set[k]
        raw_video_path = FLAGS.data_path + '/' + video_set[k]+'.mp4'
        cap = cv2.VideoCapture(raw_video_path)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        cnt_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_info.append(cnt_frame)
        # 1. load both polys and tags; 2. generate geo maps(the format of polys and tags need to match)
        polys_array_list, tags_array_list, id_list_list, frame_num = load_annotations_solo(xml_solo_path, 1, cnt_frame, frame_width, frame_height)
        polys_array_list2.append(polys_array_list)
        tags_array_list2.append(tags_array_list)
        # 2. apply RBOX box generation on the whole frame
        if FLAGS.from_source is True:
            print("Saving all frames into disk!!!")
            basename = FLAGS.pre_processed
            for m in range(cnt_frame):
                ret, image = cap.read()
                if ret is True:
                    None
                else:
                    print('Hey, we got something wrong here!')
                    continue
                # text_polys, text_tags = load_annoataion(txt_fn)
                text_polys, text_tags = polys_array_list[m], tags_array_list[m]
                text_polys, text_tags = check_and_validate_polys(text_polys, text_tags, (frame_height, frame_width))
                if text_polys.shape[0] == 0:
                    # save only the frames with boxes
                    print("there is no box on frame {}".format(m))
                    continue
                new_h, new_w = frame_height, frame_width
                score_map, geo_map, training_mask = generate_rbox((new_h, new_w), text_polys, text_tags)
                # score_name = basename + '/score_maps/' + '{}'.format(video_set[k])+ '/frame'+'{:04d}'.format(m)+'.npy'
                # geo_name = basename + '/geo_maps/' + '{}'.format(video_set[k])+'/frame'+'{:04d}'.format(m)+'.npy'
                mask_name = basename + '/training_masks/' + '{}'.format(video_set[k])+'/frame'+'{:04d}'.format(m)+'.npy'
                # np.save(score_name, score_map)
                # np.save(geo_name, geo_map)
                np.save(mask_name, training_mask)
            cap.release()
            print('Finishing video {} with size ({}, {}, {}) '.format(video_set[k], frame_width, frame_height, cnt_frame))
    images_seq = []
    score_maps_seq = []
    geo_maps_seq = []
    training_masks_seq = []
    while True:
        # random video index
        i = randint(0, len(index)-1)
        try:
            video_path = FLAGS.data_path + '/' + video_set[i]+'.mp4'
            cap = cv2.VideoCapture(video_path)
            new_h, new_w = int(cap.get(4)), int(cap.get(3))
            # data loading
            data = np.zeros([config.num_steps, new_h, new_w, 3], dtype=np.uint8)
            score_maps = np.zeros([config.num_steps, new_h, new_w], dtype=np.uint8)
            geo_maps = np.zeros([config.num_steps, new_h, new_w, 5], dtype=np.float32)
            training_masks = np.ones([config.num_steps, new_h, new_w], dtype=np.uint8)
            j = randint(0, frame_info[i]-config.num_steps-1)
            # pre-process data timing
            t1_start = time.time()
            basename = FLAGS.pre_processed
            for m in range(config.num_steps):
                cap.set(1, j+m)
                ret, image = cap.read()
                ######### add on-line data processing, Sep 5th by Xiaolong
                text_polys, text_tags = polys_array_list2[i][j+m], tags_array_list2[i][j+m]
                text_polys, text_tags = check_and_validate_polys(text_polys, text_tags, (new_h, new_w))
                if text_polys.shape[0] == 0:# means no boxes here
                    data[m, :, :, :] = image
                    continue
                # # Previously we use the
                # score_map_raw, geo_map_raw, training_mask_raw = generate_rbox((new_h, new_w), text_polys, text_tags)
                # data[m, :, :, :] = image
                # score_maps[m, :, :] = score_map_raw
                # geo_maps[m, :, :, :] = geo_map_raw
                # training_masks[m, :, :] = training_mask_raw
                assert text_polys.shape[0] != 0, "text polys is: {}".format(text_polys.shape[0])
                score_name = basename + '/score_maps/' + '{}'.format(video_set[i])+ '/frame'+'{:04d}'.format(j+m)+'.npy'
                geo_name = basename + '/geo_maps/' + '{}'.format(video_set[i])+'/frame'+'{:04d}'.format(j+m)+'.npy'
                mask_name = basename + '/training_masks/' + '{}'.format(video_set[i])+'/frame'+'{:04d}'.format(j+m)+'.npy'
                data[m, :, :, :] = image
                score_maps[m, :, :] = np.load(score_name)
                geo_maps[m, :, :, :] = np.load(geo_name)
                training_masks[m, :, :] = np.load(mask_name)
            t1_end = time.time()
            # print("pre-processing on one batch costs {} seconds".format(t1_end - t1_start))
            imgs_c, score_maps_c, geo_maps_c, training_masks_c = crop_all_random_seq(config, data, score_maps, geo_maps, training_masks)
            if FLAGS.debug is True:
                print("Cropped image has type {}".format(type(imgs_c)))
            scores_new = np.copy(score_maps_c)
            for d in range(scores_new.shape[0]):
                if (sum(scores_new[d, 0, :]) + sum(scores_new[d, 511, :]) + sum(scores_new[d, :, 0]) + sum(scores_new[d, :, 511]))>0:
                    seed_array = np.copy(scores_new[d, :, :])
                    example_array = np.copy(scores_new[d, :, :])
                    distance = nd.distance_transform_edt(example_array)
                    seed_array[1:511, 1:511] = np.zeros((510, 510))
                    result = watershed(-distance, seed_array, mask=example_array, \
                                       connectivity=square(3))
                    result_reverse = ~result
                    training_masks_c[d, :, :] = training_masks_c[d, :, :]*result_reverse
            # print("Cropping one video sequence")
            if imgs_c is not None:
                if FLAGS.debug is True:
                    print("appending data")
                images_seq.append(imgs_c.astype(np.float32))
                score_maps_seq.append(score_maps_c[:, ::4, ::4, np.newaxis].astype(np.float32))
                geo_maps_seq.append(geo_maps_c[:, ::4, ::4, :].astype(np.float32))
                training_masks_seq.append(training_masks_c[:, ::4, ::4, np.newaxis].astype(np.float32))
                if FLAGS.vis is True:
                    plt.figure(dpi=300)
                    ax = plt.subplot(121)
                    plt.imshow(score_maps_c[0, ::4, ::4])
                    plt.title("score map")
                    ax = plt.subplot(122)
                    plt.imshow(training_masks_c[0, ::4, ::4])
                    plt.title("training mask")
                    print("saving figure")
                    plt.savefig("/home/lxiaol9/debug/flow/"+str(len(geo_maps_seq))+".png")

            if len(images_seq) == config.batch_size:
                yield images_seq, score_maps_seq, geo_maps_seq, training_masks_seq
                images_seq = []
                score_maps_seq = []
                geo_maps_seq = []
                training_masks_seq = []
                # return images_seq, score_maps_seq, geo_maps_seq, training_masks_seq
        except Exception as e:
            import traceback
            traceback.print_exc()
            continue
